In [1]:
# !pip install selenium beautifulsoup4 webdriver-manager

In [2]:
# !pip install gradio

In [10]:
from bs4 import BeautifulSoup
import requests
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
import ollama
import torch
from vllm import LLM, SamplingParams
import bs4
import gradio as gr
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time


In [1]:
import json

# Define the filename
filename = "/home/maria/gpt/n/extracted_document.txt"  # Update with the actual path

# Open the file and read the contents
with open(filename, 'r', encoding='utf-8') as file:
    document = json.load(file)



In [2]:


document

{'title': 'البنك الأهلى المصرى - الصفحة الرئيسية',
 'content': 'الحسابات\nحسابات جاريةحسابات التوفيرافتح حسابك وإنت في مكانكالمزيد\n\nشهادات الادخار\nعملة محليةعملة أجنبيةشهادات بلاديشهادات الاستثمارالمزيد\n\nالبطاقات\nبطاقات الإئتمانبطاقات الخصم المباشرالبطاقات المدفوعة مقدمامحددات البطاقاتمحددات البطاقاتكيفية تفعيل البطاقاتالمزيد\n\nالقروض\nالقرض الشخصي النقديقرض السيارةمبادرة البنك المركزي المصري - التمويل العقاريالمزيد\n\nالودائع\nالودائع - عملة محليةالودائع - عملات أجنبيةالمزيد\n\nالخدمات\nالأوراق الماليةأمناء الأستثمارالتحويلات الخارجيةالتأمين البنكيالمزيد\n\nالخدمات الالكترونية\nالاهلي نت – افرادالاهلى موبايلالأهلي فون كاشفروع الخدمة الالكترونيةخدمة الأهلي للتسوق عبر الإنترنتالفرع المتنقلالأهلي واتسابخدمة المساعد الآلي التفاعلي NBE Chatbot\n\nخدمات الحسابات\nخدمة الكشف الإلكتروني للحساباتخدمة ال sweeping\n\nالحسابات \xa0و البطاقات\nحسابات جاريهبطاقات الشركاتالمزيد\n\nشهادات الادخار\nعملة محليةعملة أجنبيةالمزيد\n\nتمويل الشركات\nتمويل الشركات والقروض المشتركةالمزيد\n\nخدمات المرت

In [5]:


splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Define the size of each chunk
    chunk_overlap=100  # Define overlap between chunks
)



# Split the document's content into chunks
chunks = splitter.split_text(document['content'])


# Output the chunks
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}:")
    print(chunk)
    print("-" * 80)

Chunk 1:
الحسابات
حسابات جاريةحسابات التوفيرافتح حسابك وإنت في مكانكالمزيد

شهادات الادخار
عملة محليةعملة أجنبيةشهادات بلاديشهادات الاستثمارالمزيد

البطاقات
بطاقات الإئتمانبطاقات الخصم المباشرالبطاقات المدفوعة مقدمامحددات البطاقاتمحددات البطاقاتكيفية تفعيل البطاقاتالمزيد

القروض
القرض الشخصي النقديقرض السيارةمبادرة البنك المركزي المصري - التمويل العقاريالمزيد

الودائع
الودائع - عملة محليةالودائع - عملات أجنبيةالمزيد

الخدمات
الأوراق الماليةأمناء الأستثمارالتحويلات الخارجيةالتأمين البنكيالمزيد

الخدمات الالكترونية
الاهلي نت – افرادالاهلى موبايلالأهلي فون كاشفروع الخدمة الالكترونيةخدمة الأهلي للتسوق عبر الإنترنتالفرع المتنقلالأهلي واتسابخدمة المساعد الآلي التفاعلي NBE Chatbot

خدمات الحسابات
خدمة الكشف الإلكتروني للحساباتخدمة ال sweeping

الحسابات  و البطاقات
حسابات جاريهبطاقات الشركاتالمزيد

شهادات الادخار
عملة محليةعملة أجنبيةالمزيد

تمويل الشركات
تمويل الشركات والقروض المشتركةالمزيد

خدمات المرتبات
برامج الاهلي للمرتباتخدمة الراتب المقدمالمزيد
-----------------------------------------

In [7]:
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document

documents = [Document(page_content=chunk) for chunk in chunks]

# Load the embedding model from the SentenceTransformers library
# embedding_model = SentenceTransformer('danfeg/AraBERT_Finetuned-COMB-7443')
# Use HuggingFaceEmbeddings to wrap the model for compatibility with LangChain
# embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
# embeddings = HuggingFaceEmbeddings(model_name='danfeg/AraBERT_Finetuned-COMB-7443')
embeddings = HuggingFaceEmbeddings(model_name='danfeg/AraBERT_Finetuned-COMB-7443')

vectordb = FAISS.from_documents(documents, embeddings)

retriever = vectordb.as_retriever()



In [12]:
# Step 6: Retrieve relevant context from FAISS based on a question
def get_relevant_context(question):
    docs = retriever.get_relevant_documents(question)
    context = "\n".join([doc.page_content for doc in docs])
    return context

# Step 7: Initialize LLaMA model using vllm
llm = LLM(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct", 
    dtype=torch.bfloat16, 
    trust_remote_code=True, 
    quantization="bitsandbytes", 
    load_format="bitsandbytes", 
    gpu_memory_utilization=0.20, 
    max_model_len=5000
)


# Step 8: Prepare the prompt for the LLM
prompt_template = """\
أجب على السؤال التالي باللغة العربية باستخدام السياق المقدم فقط.
قدم إجابة واضحة وتأكد من أنها تعالج السؤال بدقة.
ابحث عن المعلومات ذات الصلة في البيانات واستخرجها.

<السياق>
{context}
</السياق>

السؤال: {input}
"""

# Step 9: Generate a response using the LLM based on retrieved context
def generate_response(question):
    # Step 9.1: Retrieve the relevant context from FAISS
    context = get_relevant_context(question)
    
    # Step 9.2: Format the prompt
    prompt_text = prompt_template.format(context=context, input=question)
    
    # Step 9.3: Generate the answer using the LLM
    sampling_params = SamplingParams(max_tokens=250, temperature=0.8)
    response = llm.generate(prompt_text, sampling_params)
    
    # Return the response text
    return response[0].outputs[0].text




WARNING 10-23 10:03:50 config.py:335] bitsandbytes quantization is not fully optimized yet. The speed can be slower than non-quantized models.
WARNING 10-23 10:03:50 config.py:430] CUDA graph is not supported on BitAndBytes yet, fallback to the eager mode.
INFO 10-23 10:03:50 llm_engine.py:232] Initializing an LLM engine (v0.6.1) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=5000, download_dir=None, load_format=LoadFormat.BITSANDBYTES, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=bitsandbytes, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='ou

INFO 10-23 10:03:50 model_runner.py:997] Starting to load model meta-llama/Meta-Llama-3.1-8B-Instruct...
INFO 10-23 10:03:50 loader.py:945] Loading weights with BitsAndBytes quantization.  May take a while ...
INFO 10-23 10:03:50 weight_utils.py:242] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 10-23 10:03:53 model_runner.py:1008] Loading model weights took 5.3109 GB
INFO 10-23 10:03:54 gpu_executor.py:122] # GPU blocks: 970, # CPU blocks: 2048


Processed prompts: 100%|██████████| 1/1 [00:09<00:00,  9.58s/it, est. speed input: 170.11 toks/s, output: 26.11 toks/s]

الإجابة: البنك الأهلي المصري هو أحد البنوك الرئيسية في مصر، ويعمل مجهودا كبيرا من أجل تقديم الخدمات المصرفية المتطورة وتحقيق الاستدامة المالية. بالإضافة إلى ذلك، فإن البنك لديه فرع في عديس أبابا. البنك يوفر مجموعة واسعة من الخدمات في مجال التمويل والهيئات الأخرى، بما في ذلك شهادات الوديعة والودائع - عملة محلية. يصنف البنك ضمن القطاع العام المصرى. الاستثمار البنكى للإستثمار المباشر في الأسهم المصرفية هو عنصر أساسي يعتمد عليه الأسواق المالية، ويقدم البنك استشارات وفرص إستثمار. يعتبر البنك النموذج المصاحب للιχتراق الاقتصادى المصرى، مما يجعله اسسيا للامنات في مصر. يعتمد البنك على عدد من العوامل في تقديم العروض والطلبات على الأسهم المصرية
